In [1]:
import pandas as pd  
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
evals= pd.DataFrame(pd.read_excel('../data/bdi3_all_evals_clean.xlsx'))
evals.head()

,Child ID,Gender,Date of Birth,Location - Sub Level 1,Program Label,Adaptive Sum of Scaled Scores,Adaptive Developmental Quotient,Adaptive Percentile Rank,Adaptive 95% Confidence Interval,Adaptive RDI,...,Cognitive-Perception and Concepts RS,Cognitive-Perception and Concepts SS,Cognitive-Perception and Concepts PR,Cognitive-Perception and Concepts AE,Cognitive-Perception and Concepts RDI,Cognitive-Perception and Concepts CSS,Cognitive-Perception and Concepts CSS 90%,Cognitive-Perception and Concepts Z-Score,Cognitive-Perception and Concepts T-Score,Cognitive-Perception and Concepts NCE
0,44879,F,2022-09-27,Southwest,BDI-3 Eligibility Evaluation,7.0,85.0,16,77-97,39/90,...,9.0,13.0,84,12,98/90,411.0,392 - 431,1,60.0,71
1,47795,F,2020-12-23,Greater Nashville,BDI-3 Eligibility Evaluation,16.0,88.0,21,82-96,79/90,...,11.0,7.0,16,18,29/90,437.0,421 - 453,– 1.00,40.0,29
2,54340,M,2021-06-22,First Tennessee,BDI-3 Eligibility Evaluation,8.0,90.0,25,81-101,77/90,...,10.0,7.0,16,15,37/90,426.0,408 - 444,– 1.00,40.0,29
3,54344,F,2022-03-07,Southeast Tennessee,BDI-3 Eligibility Evaluation,13.0,115.0,84,103-123,99/90,...,9.0,10.0,50,12,92/90,411.0,392 - 431,0,50.0,50
4,54515,M,2021-11-04,Memphis Delta,BDI-3 Eligibility Evaluation,5.0,75.0,5,68-88,1990-09-01 00:00:00,...,10.0,10.0,50,15,88/90,426.0,408 - 444,0,50.0,50


In [3]:
evals.value_counts('Program Label')

Program Label
BDI-3 Eligibility Evaluation          12403
BDI-3 Annual Evaluation               11205
BDI-3 Milestone or Exit Evaluation      339
Name: count, dtype: int64

In [4]:
evals.value_counts('Location - Sub Level 1')

Location - Sub Level 1
Greater Nashville      4262
East Tennessee         3866
Memphis Delta          3309
South Central          3287
Upper Cumberland       2614
First Tennessee        2045
Southeast Tennessee    1873
Southwest              1352
Northwest              1207
Name: count, dtype: int64

In [5]:
adaptive_scores = evals.groupby(['Program Label'])['Adaptive Developmental Quotient'].mean()
comm_scores = evals.groupby(['Program Label'])['Communication Developmental Quotient'].mean()
motor_scores = evals.groupby(['Program Label'])['Motor Developmental Quotient'].mean()
cog_scores = evals.groupby(['Program Label'])['Cognitive Developmental Quotient'].mean()
BDI3_total = evals.groupby(['Program Label'])['BDI-3 Total Developmental Quotient'].mean()

In [6]:
domain_dev_quo = pd.DataFrame({'Adaptive': adaptive_scores, 'Communication': comm_scores, 'Motor': motor_scores, 'Cognitive': cog_scores, 'BDI3 Total Developmental Quotient': BDI3_total})
domain_dev_quo

,Adaptive,Communication,Motor,Cognitive,BDI3 Total Developmental Quotient
Program Label,,,,,
BDI-3 Annual Evaluation,84.604570,76.821106,90.875648,85.661963,84.951853
BDI-3 Eligibility Evaluation,89.184238,81.181554,96.569629,91.002502,89.299264
BDI-3 Milestone or Exit Evaluation,85.460177,75.530973,94.433628,84.578171,85.870206
